# Lab 4.6.3: Option B - Multimodal Document Intelligence

**Module:** 4.6 - Capstone Project (Domain 4: Production AI)
**Time:** 35-45 hours total
**Difficulty:** ⭐⭐⭐⭐⭐

---

## 🎯 Project Overview

Build a **document intelligence system** that processes and understands complex documents:

| Component | Description | DGX Spark Advantage |
|-----------|-------------|---------------------|
| **Document Ingestion** | PDF, images, diagrams | High-memory batch processing |
| **Vision-Language Model** | LLaVA/Qwen2-VL 34B | Full-resolution VLM fits in memory |
| **OCR & Layout** | Text extraction with structure | GPU-accelerated OCR |
| **Information Extraction** | Structured data from unstructured | Multi-model pipeline |
| **Multimodal RAG** | Q&A over documents | VLM + embeddings together |
| **Export Pipeline** | JSON, CSV, structured output | Fast batch processing |

---

## 🎯 Learning Objectives

By completing this project, you will:
- [ ] Build a document ingestion pipeline (PDF, images)
- [ ] Integrate vision-language models for understanding
- [ ] Extract structured information from documents
- [ ] Create a multimodal RAG system
- [ ] Build an interactive document Q&A demo
- [ ] Implement content safety filtering 🛡️

---

## 🌍 Real-World Context

Document intelligence is transforming how organizations handle paperwork:

| Industry | Use Case | Value |
|----------|----------|-------|
| **Finance** | Invoice processing, contract analysis | 90% faster processing |
| **Healthcare** | Medical records, lab reports | Reduced transcription errors |
| **Legal** | Contract review, discovery | Hours → minutes for review |
| **Insurance** | Claims processing | Automated damage assessment |
| **Research** | Paper summarization | Rapid literature review |

Companies like Google (Document AI), AWS (Textract), and startups like Reducto are building billion-dollar businesses around document understanding.

---

## 🧒 ELI5: What is Document Intelligence?

> **Imagine you have a huge stack of papers** - invoices, contracts, reports, handwritten notes.
>
> Reading them all would take forever. But what if you had a super-smart assistant who could:
>
> 1. **Read everything** - even messy handwriting and blurry photos
> 2. **Understand what they mean** - not just the words, but the context
> 3. **Find specific information** - "What's the total on invoice #1234?"
> 4. **Organize it all** - put data into neat spreadsheets
> 5. **Answer questions** - "Which contracts expire next month?"
>
> **That's document intelligence!** It combines:
> - **OCR** (reading text from images)
> - **Vision models** (understanding layouts and diagrams)
> - **Language models** (understanding meaning)
> - **RAG** (finding and retrieving information)
>
> The result? Turn hours of manual work into seconds of AI processing.

---

## 🏗️ System Architecture

```
┌─────────────────────────────────────────────────────────────────────────┐
│                    Multimodal Document Intelligence                      │
├─────────────────────────────────────────────────────────────────────────┤
│                                                                          │
│  ┌──────────────┐    ┌──────────────┐    ┌──────────────┐              │
│  │   Gradio     │───▶│    Query     │───▶│   Response   │              │
│  │  Interface   │    │   Router     │    │   Builder    │              │
│  └──────────────┘    └───────┬──────┘    └──────────────┘              │
│                              │                                          │
│         ┌────────────────────┼────────────────────┐                     │
│         ▼                    ▼                    ▼                     │
│  ┌──────────────┐    ┌──────────────┐    ┌──────────────┐              │
│  │  Multimodal  │    │   Vision-    │    │  Structured  │              │
│  │     RAG      │    │   Language   │    │  Extraction  │              │
│  │              │    │    Model     │    │              │              │
│  └───────┬──────┘    └──────────────┘    └───────┬──────┘              │
│          │                                       │                      │
│  ┌───────┴──────┐                        ┌───────┴──────┐              │
│  │ Vector Store │                        │   Schema     │              │
│  │ (text+image) │                        │   Library    │              │
│  └───────┬──────┘                        └──────────────┘              │
│          │                                                              │
│  ┌───────┴──────────────────────────────────────────────┐              │
│  │              Document Processing Pipeline             │              │
│  │  ┌────────┐  ┌────────┐  ┌────────┐  ┌────────┐     │              │
│  │  │  PDF   │─▶│  OCR   │─▶│ Layout │─▶│ Chunk  │     │              │
│  │  │ Parser │  │ Engine │  │ Detect │  │  +Embed│     │              │
│  │  └────────┘  └────────┘  └────────┘  └────────┘     │              │
│  └──────────────────────────────────────────────────────┘              │
│                                                                          │
│  Memory Budget: ~55-65GB of 128GB                                       │
└─────────────────────────────────────────────────────────────────────────┘
```

---

## Part 1: Environment Setup

In [ ]:
# Environment Setup
import os
import sys
from pathlib import Path
from datetime import datetime
from typing import List, Dict, Any, Optional, Union
from dataclasses import dataclass, field
import json

import torch

print("🔍 OPTION B: MULTIMODAL DOCUMENT INTELLIGENCE")
print("="*70)
print(f"Date: {datetime.now().strftime('%Y-%m-%d %H:%M')}")
print(f"\nGPU: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'Not available'}")
print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
print(f"CUDA Version: {torch.version.cuda}")
print(f"PyTorch Version: {torch.__version__}")

# Check for vision libraries
print("\n📦 Vision Libraries:")
libs = [
    ("PIL", "pillow"),
    ("pdf2image", "pdf2image"),
    ("pytesseract", "pytesseract"),
    ("transformers", "transformers"),
]
for import_name, package_name in libs:
    try:
        __import__(import_name)
        print(f"   ✅ {package_name}")
    except ImportError:
        print(f"   ❌ {package_name} - pip install {package_name}")

In [ ]:
# Memory Planning for Document Intelligence

def plan_option_b_memory():
    """Plan memory usage for document intelligence system."""
    
    components = [
        {"name": "Qwen2-VL 7B (BF16)", "memory_gb": 14.0, "alt": "Or LLaVA-1.6 34B: 18GB INT4"},
        {"name": "CLIP/SigLIP Embeddings", "memory_gb": 2.0, "alt": ""},
        {"name": "Text Embeddings (BGE-M3)", "memory_gb": 1.2, "alt": ""},
        {"name": "OCR Model (optional)", "memory_gb": 1.0, "alt": "TrOCR or PaddleOCR"},
        {"name": "Layout Detection", "memory_gb": 0.5, "alt": "DiT or LayoutLM"},
        {"name": "Vector Index (FAISS)", "memory_gb": 2.0, "alt": ""},
        {"name": "Image Processing Buffer", "memory_gb": 8.0, "alt": "For batch processing"},
        {"name": "KV Cache", "memory_gb": 6.0, "alt": ""},
        {"name": "System Overhead", "memory_gb": 6.0, "alt": ""},
    ]
    
    DGX_SPARK_MEMORY = 128.0
    
    print("\n💾 DGX SPARK MEMORY PLAN - OPTION B")
    print("="*70)
    
    total = 0
    for comp in components:
        alt_str = f" ({comp['alt']})" if comp['alt'] else ""
        print(f"   {comp['name']:<30} {comp['memory_gb']:>6.1f} GB{alt_str}")
        total += comp['memory_gb']
    
    print("-"*70)
    print(f"   {'TOTAL':<30} {total:>6.1f} GB")
    print(f"   {'REMAINING':<30} {DGX_SPARK_MEMORY - total:>6.1f} GB")
    
    # For larger VLM option
    print("\n📊 Alternative: LLaVA-1.6 34B Configuration")
    print(f"   VLM (34B INT4): ~18 GB")
    print(f"   Total with 34B: ~{total + 4:.1f} GB")
    print(f"   ✅ Still fits comfortably!")
    
    return total

plan_option_b_memory()

---

## Part 2: Document Processing Pipeline

In [ ]:
# Document Processing Classes

@dataclass
class DocumentPage:
    """A single page from a document."""
    page_number: int
    image: Any  # PIL Image
    text: str = ""
    layout: Dict[str, Any] = field(default_factory=dict)
    tables: List[Dict] = field(default_factory=list)
    figures: List[Dict] = field(default_factory=list)

@dataclass
class ProcessedDocument:
    """A fully processed document."""
    id: str
    filename: str
    pages: List[DocumentPage]
    metadata: Dict[str, Any]
    full_text: str = ""
    extracted_data: Dict[str, Any] = field(default_factory=dict)

class DocumentProcessor:
    """
    Process documents (PDF, images) for the intelligence system.
    
    Features:
    - PDF to image conversion
    - OCR with layout preservation
    - Table and figure detection
    - Text chunking for RAG
    
    Example:
        processor = DocumentProcessor()
        doc = processor.process("invoice.pdf")
        print(doc.full_text)
    """
    
    def __init__(
        self,
        ocr_engine: str = "pytesseract",
        dpi: int = 200,
        use_gpu: bool = True,
    ):
        self.ocr_engine = ocr_engine
        self.dpi = dpi
        self.use_gpu = use_gpu
        self._doc_id_counter = 0
    
    def process(self, file_path: str) -> ProcessedDocument:
        """
        Process a document file.
        
        Supports: PDF, PNG, JPG, TIFF
        """
        file_path = Path(file_path)
        
        if not file_path.exists():
            raise FileNotFoundError(f"Document not found: {file_path}")
        
        self._doc_id_counter += 1
        doc_id = f"doc_{self._doc_id_counter:05d}"
        
        # Convert to images
        if file_path.suffix.lower() == ".pdf":
            images = self._pdf_to_images(file_path)
        else:
            from PIL import Image
            images = [Image.open(file_path)]
        
        # Process each page
        pages = []
        all_text = []
        
        for i, img in enumerate(images):
            page = self._process_page(img, i + 1)
            pages.append(page)
            all_text.append(page.text)
        
        return ProcessedDocument(
            id=doc_id,
            filename=file_path.name,
            pages=pages,
            metadata={
                "source": str(file_path),
                "num_pages": len(pages),
                "processed_at": datetime.now().isoformat(),
            },
            full_text="\n\n".join(all_text)
        )
    
    def _pdf_to_images(self, pdf_path: Path) -> List:
        """Convert PDF to list of images."""
        try:
            from pdf2image import convert_from_path
            return convert_from_path(pdf_path, dpi=self.dpi)
        except ImportError:
            print("⚠️ pdf2image not installed. Install with: pip install pdf2image")
            print("   Also requires poppler: apt-get install poppler-utils")
            return []
    
    def _process_page(self, image, page_num: int) -> DocumentPage:
        """Process a single page image."""
        # OCR
        text = self._ocr(image)
        
        # Layout detection (simplified)
        layout = self._detect_layout(image)
        
        return DocumentPage(
            page_number=page_num,
            image=image,
            text=text,
            layout=layout,
        )
    
    def _ocr(self, image) -> str:
        """Extract text from image using OCR."""
        try:
            import pytesseract
            return pytesseract.image_to_string(image)
        except ImportError:
            # Fallback: use VLM for OCR (in production)
            return "[OCR not available - install pytesseract]"
    
    def _detect_layout(self, image) -> Dict:
        """Detect document layout (simplified)."""
        # In production, use LayoutLM or DiT
        return {
            "width": image.width if hasattr(image, 'width') else 0,
            "height": image.height if hasattr(image, 'height') else 0,
            "detected_regions": [],
        }

print("✅ Document processing classes defined")

---

## Part 3: Vision-Language Model Integration

In [ ]:
# Vision-Language Model Wrapper

class VisionLanguageModel:
    """
    Vision-Language Model for document understanding.
    
    Supports:
    - Qwen2-VL (7B, 72B)
    - LLaVA-1.6 (7B, 13B, 34B)
    - InternVL
    
    Example:
        vlm = VisionLanguageModel("Qwen/Qwen2-VL-7B-Instruct")
        result = vlm.analyze(image, "What information is on this invoice?")
    """
    
    SUPPORTED_MODELS = {
        "qwen2-vl-7b": "Qwen/Qwen2-VL-7B-Instruct",
        "qwen2-vl-72b": "Qwen/Qwen2-VL-72B-Instruct",
        "llava-1.6-7b": "llava-hf/llava-v1.6-mistral-7b-hf",
        "llava-1.6-34b": "llava-hf/llava-v1.6-34b-hf",
    }
    
    def __init__(
        self,
        model_name: str = "qwen2-vl-7b",
        load_in_4bit: bool = False,
        max_image_size: int = 1024,
    ):
        self.model_id = self.SUPPORTED_MODELS.get(model_name, model_name)
        self.load_in_4bit = load_in_4bit
        self.max_image_size = max_image_size
        self._model = None
        self._processor = None
    
    def load(self):
        """Load the VLM into memory."""
        if self._model is not None:
            return
        
        from transformers import AutoProcessor, AutoModelForVision2Seq
        from transformers import BitsAndBytesConfig
        
        print(f"📥 Loading VLM: {self.model_id}")
        
        if self.load_in_4bit:
            bnb_config = BitsAndBytesConfig(
                load_in_4bit=True,
                bnb_4bit_compute_dtype=torch.bfloat16,
            )
            self._model = AutoModelForVision2Seq.from_pretrained(
                self.model_id,
                quantization_config=bnb_config,
                device_map="auto",
                trust_remote_code=True,
            )
        else:
            self._model = AutoModelForVision2Seq.from_pretrained(
                self.model_id,
                torch_dtype=torch.bfloat16,
                device_map="auto",
                trust_remote_code=True,
            )
        
        self._processor = AutoProcessor.from_pretrained(
            self.model_id,
            trust_remote_code=True,
        )
        
        mem_gb = torch.cuda.memory_allocated() / 1e9
        print(f"✅ VLM loaded. GPU memory: {mem_gb:.2f} GB")
    
    def analyze(
        self,
        image,
        prompt: str,
        max_tokens: int = 1024,
    ) -> str:
        """
        Analyze an image with a text prompt.
        
        Args:
            image: PIL Image or path to image
            prompt: Question or instruction
            max_tokens: Maximum response length
            
        Returns:
            Model's response
        """
        self.load()
        
        from PIL import Image
        
        # Load image if path
        if isinstance(image, (str, Path)):
            image = Image.open(image)
        
        # Resize if needed
        if max(image.size) > self.max_image_size:
            ratio = self.max_image_size / max(image.size)
            new_size = (int(image.width * ratio), int(image.height * ratio))
            image = image.resize(new_size)
        
        # Process
        inputs = self._processor(
            text=prompt,
            images=image,
            return_tensors="pt",
        ).to("cuda")
        
        with torch.no_grad():
            outputs = self._model.generate(
                **inputs,
                max_new_tokens=max_tokens,
                do_sample=True,
                temperature=0.7,
            )
        
        response = self._processor.decode(
            outputs[0],
            skip_special_tokens=True,
        )
        
        return response
    
    def extract_structured(
        self,
        image,
        schema: Dict[str, str],
    ) -> Dict:
        """
        Extract structured data from document image.
        
        Args:
            image: Document image
            schema: Dict of {field_name: description}
            
        Returns:
            Dict with extracted values
        """
        schema_text = "\n".join([
            f"- {name}: {desc}" for name, desc in schema.items()
        ])
        
        prompt = f"""Extract the following information from this document image.
Return the result as a JSON object.

Fields to extract:
{schema_text}

Return ONLY the JSON object, no other text."""
        
        response = self.analyze(image, prompt)
        
        # Parse JSON from response
        try:
            # Find JSON in response
            import re
            json_match = re.search(r'\{[^{}]*\}', response, re.DOTALL)
            if json_match:
                return json.loads(json_match.group())
        except json.JSONDecodeError:
            pass
        
        return {"raw_response": response}
    
    def unload(self):
        """Unload model to free memory."""
        if self._model is not None:
            del self._model
            del self._processor
            self._model = None
            self._processor = None
            torch.cuda.empty_cache()
            print("✅ VLM unloaded")

print("✅ VisionLanguageModel class defined")
print(f"\nSupported models: {list(VisionLanguageModel.SUPPORTED_MODELS.keys())}")

---

## Part 4: Information Extraction Schemas

In [ ]:
# Document Extraction Schemas

EXTRACTION_SCHEMAS = {
    "invoice": {
        "vendor_name": "Name of the company issuing the invoice",
        "invoice_number": "Unique invoice identifier",
        "invoice_date": "Date the invoice was issued (YYYY-MM-DD)",
        "due_date": "Payment due date (YYYY-MM-DD)",
        "total_amount": "Total amount due (number only)",
        "currency": "Currency code (USD, EUR, etc.)",
        "line_items": "List of items with description, quantity, and price",
        "payment_terms": "Payment terms or conditions",
    },
    "receipt": {
        "merchant_name": "Store or business name",
        "date": "Transaction date",
        "total": "Total amount paid",
        "payment_method": "How payment was made (cash, card, etc.)",
        "items": "List of purchased items",
    },
    "contract": {
        "parties": "Names of parties involved",
        "effective_date": "When the contract takes effect",
        "expiration_date": "When the contract expires",
        "contract_type": "Type of agreement",
        "key_terms": "Important terms and conditions",
        "signatures": "Names of signatories",
    },
    "research_paper": {
        "title": "Paper title",
        "authors": "List of authors",
        "abstract": "Paper abstract or summary",
        "keywords": "Key topics or keywords",
        "publication": "Journal or conference name",
        "year": "Publication year",
    },
    "form": {
        "form_type": "Type of form",
        "filled_fields": "All filled-in fields with their values",
        "checkboxes": "Status of any checkboxes",
        "signatures": "Any signatures present",
    },
}

class StructuredExtractor:
    """
    Extract structured data from documents using schemas.
    
    Example:
        extractor = StructuredExtractor(vlm)
        data = extractor.extract(image, "invoice")
    """
    
    def __init__(self, vlm: VisionLanguageModel):
        self.vlm = vlm
        self.schemas = EXTRACTION_SCHEMAS
    
    def extract(
        self,
        image,
        document_type: str,
        custom_schema: Dict[str, str] = None,
    ) -> Dict:
        """
        Extract data using predefined or custom schema.
        
        Args:
            image: Document image
            document_type: Key in EXTRACTION_SCHEMAS or "custom"
            custom_schema: Custom extraction schema
            
        Returns:
            Extracted data matching schema
        """
        if custom_schema:
            schema = custom_schema
        elif document_type in self.schemas:
            schema = self.schemas[document_type]
        else:
            raise ValueError(f"Unknown document type: {document_type}. "
                           f"Available: {list(self.schemas.keys())}")
        
        result = self.vlm.extract_structured(image, schema)
        result["_document_type"] = document_type
        result["_extracted_at"] = datetime.now().isoformat()
        
        return result
    
    def batch_extract(
        self,
        documents: List[ProcessedDocument],
        document_type: str,
    ) -> List[Dict]:
        """Extract from multiple documents."""
        results = []
        for doc in documents:
            for page in doc.pages:
                result = self.extract(page.image, document_type)
                result["_document_id"] = doc.id
                result["_page"] = page.page_number
                results.append(result)
        return results

print("✅ Extraction schemas defined")
print(f"\nAvailable document types: {list(EXTRACTION_SCHEMAS.keys())}")

---

## Part 5: Multimodal RAG System

In [ ]:
# Multimodal RAG for Document Q&A

@dataclass
class MultimodalChunk:
    """A chunk that can contain text and/or image."""
    id: str
    text: str
    image: Optional[Any] = None  # PIL Image
    embedding: Optional[List[float]] = None
    metadata: Dict[str, Any] = field(default_factory=dict)

class MultimodalRAG:
    """
    RAG system for multimodal document Q&A.
    
    Combines:
    - Text embeddings (for text content)
    - Image embeddings (for visual content)
    - VLM for answer generation
    
    Example:
        rag = MultimodalRAG(vlm)
        rag.add_document(processed_doc)
        answer = rag.query("What is the total on invoice #1234?")
    """
    
    def __init__(
        self,
        vlm: VisionLanguageModel,
        text_embedding_model: str = "BAAI/bge-m3",
        chunk_size: int = 500,
    ):
        self.vlm = vlm
        self.text_embedding_model = text_embedding_model
        self.chunk_size = chunk_size
        self.chunks: List[MultimodalChunk] = []
        self.text_index = None
        self._text_embedder = None
        self._chunk_id_counter = 0
    
    def _load_text_embedder(self):
        """Load text embedding model."""
        if self._text_embedder is None:
            from sentence_transformers import SentenceTransformer
            self._text_embedder = SentenceTransformer(
                self.text_embedding_model,
                device="cuda"
            )
        return self._text_embedder
    
    def add_document(self, doc: ProcessedDocument):
        """
        Add a processed document to the RAG system.
        """
        import numpy as np
        
        embedder = self._load_text_embedder()
        new_chunks = []
        
        for page in doc.pages:
            # Create chunks from page text
            text_chunks = self._chunk_text(page.text)
            
            for i, text in enumerate(text_chunks):
                self._chunk_id_counter += 1
                chunk = MultimodalChunk(
                    id=f"chunk_{self._chunk_id_counter}",
                    text=text,
                    image=page.image if i == 0 else None,  # First chunk gets image
                    metadata={
                        "document_id": doc.id,
                        "filename": doc.filename,
                        "page": page.page_number,
                        "chunk_index": i,
                    }
                )
                new_chunks.append(chunk)
        
        # Generate embeddings
        texts = [c.text for c in new_chunks]
        embeddings = embedder.encode(
            texts,
            normalize_embeddings=True,
            show_progress_bar=True,
        )
        
        for chunk, emb in zip(new_chunks, embeddings):
            chunk.embedding = emb.tolist()
        
        self.chunks.extend(new_chunks)
        
        # Rebuild index
        self._build_index()
        
        print(f"✅ Added {len(new_chunks)} chunks from {doc.filename}")
    
    def _chunk_text(self, text: str) -> List[str]:
        """Split text into chunks."""
        words = text.split()
        chunks = []
        
        for i in range(0, len(words), self.chunk_size):
            chunk = " ".join(words[i:i + self.chunk_size])
            if len(chunk) > 50:
                chunks.append(chunk)
        
        return chunks if chunks else [text]  # At least one chunk
    
    def _build_index(self):
        """Build FAISS index."""
        import faiss
        import numpy as np
        
        embeddings = np.array([
            c.embedding for c in self.chunks if c.embedding
        ]).astype('float32')
        
        if len(embeddings) == 0:
            return
        
        dim = embeddings.shape[1]
        self.text_index = faiss.IndexFlatIP(dim)
        self.text_index.add(embeddings)
    
    def query(
        self,
        question: str,
        top_k: int = 3,
        include_images: bool = True,
    ) -> Dict:
        """
        Query the document collection.
        
        Args:
            question: User's question
            top_k: Number of chunks to retrieve
            include_images: Whether to use VLM on retrieved images
            
        Returns:
            Dict with answer, sources, and confidence
        """
        import time
        import numpy as np
        
        start = time.time()
        
        # Retrieve relevant chunks
        embedder = self._load_text_embedder()
        query_emb = embedder.encode([question], normalize_embeddings=True)
        
        scores, indices = self.text_index.search(
            query_emb.astype('float32'), top_k
        )
        
        retrieved = []
        for score, idx in zip(scores[0], indices[0]):
            if idx >= 0:
                retrieved.append({
                    "chunk": self.chunks[idx],
                    "score": float(score),
                })
        
        # Build context
        context_parts = []
        images = []
        
        for r in retrieved:
            chunk = r["chunk"]
            context_parts.append(f"[Source: {chunk.metadata.get('filename', 'unknown')} "
                                f"Page {chunk.metadata.get('page', '?')}]\n{chunk.text}")
            if chunk.image and include_images:
                images.append(chunk.image)
        
        context = "\n\n---\n\n".join(context_parts)
        
        # Generate answer
        if images and include_images:
            # Use VLM with first relevant image
            prompt = f"""Based on this document image and the following context, answer the question.

Context from documents:
{context}

Question: {question}

Provide a clear, concise answer based on the document content."""
            
            answer = self.vlm.analyze(images[0], prompt)
        else:
            # Text-only answer
            prompt = f"""Based on the following context, answer the question.

Context:
{context}

Question: {question}

Answer:"""
            # Use text model or VLM without image
            answer = f"Based on the documents: {context[:500]}..."  # Placeholder
        
        latency_ms = (time.time() - start) * 1000
        
        return {
            "answer": answer,
            "sources": [
                {
                    "filename": r["chunk"].metadata.get("filename"),
                    "page": r["chunk"].metadata.get("page"),
                    "score": r["score"],
                    "text_preview": r["chunk"].text[:200],
                }
                for r in retrieved
            ],
            "latency_ms": latency_ms,
            "images_used": len(images) if include_images else 0,
        }

print("✅ MultimodalRAG class defined")

---

## Part 6: Safety & Content Filtering 🛡️

In [ ]:
# Document Content Safety

class DocumentSafetyFilter:
    """
    Safety filter for document processing.
    
    Checks for:
    - PII in extracted text (SSN, credit cards, etc.)
    - Sensitive document types
    - Malicious file content
    """
    
    def __init__(self):
        self.pii_patterns = {
            "ssn": r"\b\d{3}-\d{2}-\d{4}\b",
            "credit_card": r"\b\d{4}[- ]?\d{4}[- ]?\d{4}[- ]?\d{4}\b",
            "email": r"\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b",
            "phone": r"\b\d{3}[-.]?\d{3}[-.]?\d{4}\b",
            "bank_account": r"\b\d{8,17}\b",  # Common account number length
        }
        
        self.sensitive_keywords = [
            "confidential", "classified", "top secret",
            "internal only", "do not distribute",
            "attorney-client privilege", "hipaa",
        ]
    
    def check_text(self, text: str) -> Dict:
        """
        Check text for PII and sensitive content.
        
        Returns:
            Dict with findings and recommendations
        """
        import re
        
        findings = {
            "pii_detected": [],
            "sensitive_markers": [],
            "is_safe": True,
            "recommendations": [],
        }
        
        text_lower = text.lower()
        
        # Check PII patterns
        for pii_type, pattern in self.pii_patterns.items():
            matches = re.findall(pattern, text)
            if matches:
                findings["pii_detected"].append({
                    "type": pii_type,
                    "count": len(matches),
                })
                findings["is_safe"] = False
        
        # Check sensitive keywords
        for keyword in self.sensitive_keywords:
            if keyword in text_lower:
                findings["sensitive_markers"].append(keyword)
        
        # Generate recommendations
        if findings["pii_detected"]:
            findings["recommendations"].append(
                "Redact or mask PII before sharing extracted data"
            )
        
        if findings["sensitive_markers"]:
            findings["recommendations"].append(
                "Document may contain confidential information - verify authorization"
            )
        
        return findings
    
    def redact_pii(self, text: str) -> str:
        """
        Redact PII from text.
        
        Returns:
            Text with PII replaced by [REDACTED]
        """
        import re
        
        redacted = text
        
        for pii_type, pattern in self.pii_patterns.items():
            redacted = re.sub(pattern, f"[{pii_type.upper()}_REDACTED]", redacted)
        
        return redacted

# Test safety filter
filter = DocumentSafetyFilter()

test_text = """
Invoice for John Smith
SSN: 123-45-6789
Credit Card: 4111 1111 1111 1111
Email: john.smith@example.com
CONFIDENTIAL - Internal Use Only
"""

print("🛡️ DOCUMENT SAFETY FILTER TEST")
print("="*70)
result = filter.check_text(test_text)
print(f"\nSafe: {result['is_safe']}")
print(f"PII Found: {result['pii_detected']}")
print(f"Sensitive Markers: {result['sensitive_markers']}")
print(f"\nRecommendations:")
for rec in result['recommendations']:
    print(f"  • {rec}")

print(f"\n📝 Redacted Version:")
print(filter.redact_pii(test_text))

---

## 🎉 Checkpoint

You now have the components for a document intelligence system:

- ✅ Document processing pipeline (PDF, images)
- ✅ Vision-Language Model integration
- ✅ Structured extraction with schemas
- ✅ Multimodal RAG for Q&A
- ✅ Safety filtering for PII 🛡️

---

## 🚀 Implementation Roadmap

### Week 1: Document Pipeline
- [ ] Set up PDF processing (poppler, pdf2image)
- [ ] Configure OCR (Tesseract or TrOCR)
- [ ] Test with sample documents

### Week 2: VLM & Extraction
- [ ] Load and test VLM (Qwen2-VL or LLaVA)
- [ ] Create extraction schemas for your document types
- [ ] Build structured extraction pipeline

### Week 3: RAG & Q&A
- [ ] Build multimodal RAG system
- [ ] Test retrieval quality
- [ ] Implement answer generation

### Week 4: Integration
- [ ] Build end-to-end pipeline
- [ ] Add safety filters
- [ ] Create export functionality

### Week 5: Evaluation
- [ ] Benchmark extraction accuracy
- [ ] Test Q&A quality
- [ ] Measure latency and throughput

### Week 6: Demo & Docs
- [ ] Build Gradio demo
- [ ] Complete technical report
- [ ] Record demo video

In [ ]:
# 🧹 Cleanup
import gc

if torch.cuda.is_available():
    torch.cuda.empty_cache()
gc.collect()

print("✅ Cleanup complete!")
print("\n🎯 Next: Review examples/option-b-document-intelligence/ for complete implementation")